# FABRIC Experimenters Workshop: Directional Latency Demo

## Experiment Description

<img src="./figs/Latency-DemoExperiment.png" width="40%"><br>



## Configure the Environment

### Create the FABRIC Proxies

In [ ]:
import os
import json

from fabric_cm.credmgr.credmgr_proxy import CredmgrProxy
from fabric_cf.orchestrator.orchestrator_proxy import OrchestratorProxy

#Create a FABRIC Orchestrator Proxy.
orchestrator_proxy = OrchestratorProxy(orchestrator_host=os.environ['FABRIC_ORCHESTRATOR_HOST'])

#Create a Credential Manager Proxy 
credmgr_proxy = CredmgrProxy(credmgr_host=os.environ['FABRIC_CREDMGR_HOST'])

#Cofigure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()
    #ssh_key=ssh_key.strip()

### Set the Refresh Token

In [ ]:
#Retrieve or set the refresh token (exprires 24 hours after login)
fabric_refresh_token=None
%store -r fabric_refresh_token
if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token

print("Refresh Token: {}".format(fabric_refresh_token))

### Refresh the ID Token

ID Tokens exprire one hour after refresh. 

In [ ]:
try:
    refresh_res = credmgr_proxy.refresh(project_name='all', scope='all', refresh_token=fabric_refresh_token)

    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print("New Refreash Token: {}\n".format(fabric_refresh_token))
    print("New ID Token: {}".format(fabric_id_token))
    %store fabric_refresh_token

except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))
    


## Build Slice Request


<img src="./figs/Latency-DemoTopo.png" width="95%"><br>



In [ ]:
import fim.user as fu

# Create Experiment Topology
experiemnt = fu.ExperimentTopology()

slice_name="LatencyDemo"

In [ ]:
uky1-demo1 = experiemnt.add_node(name='uky1-demo1', site='UKY')

uky1-demo1_capacity = fu.Capacities()
uky1-demo1_capacity.set_fields(core=4, ram=32, disk=500)
uky1-demo1.set_properties(capacities=uky1-demo1_capacity, image_type='qcow2', image_ref='default_ubuntu_20')

uky1-demo1_nic = uky1-demo1.add_component(ctype=fu.ComponentType.SharedNIC, model='ConnectX-6', name='uky1-demo1_nic')

In [ ]:
renc1-demo1 = experiemnt.add_node(name='renc1-demo1', site='RENC')

renc1-demo1_capacity = fu.Capacities()
renc1-demo1_capacity.set_fields(core=4, ram=32, disk=500)
renc1-demo1.set_properties(capacities=renc1-demo1_capacity, image_type='qcow2', image_ref='default_ubuntu_20')

renc1-demo1_nic  = renc1-demo1.add_component(ctype=fu.ComponentType.SharedNIC, model='ConnectX-6', name='renc1-demo1_nic')

In [ ]:
uky2-demo1 = experiemnt.add_node(name='uky2-demo1', site='UKY')

uky2-demo1_capacity = fu.Capacities()
uky2-demo1_capacity.set_fields(core=4, ram=32, disk=500)
uky2-demo1.set_properties(capacities=uky2-demo1_capacity, image_type='qcow2', image_ref='default_ubuntu_20')

uky2-demo1_nic  = uky2-demo1.add_component(ctype=fu.ComponentType.SharedNIC, model='ConnectX-6', name='uky2-demo1_nic')

In [ ]:
# Generate Slice Graph
#slice_graph = t.serialize()
experiemnt.draw()

## Submit the Request

In [ ]:
slice_graph = experiemnt.serialize()

# Request slice from Orchestrator
status, reservations = orchestrator_proxy.create(token=fabric_id_token, slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key)


print("Response Status {}".format(status))
slice_id=reservations[0].slice_id

print("Reservations: {}".format(reservations))
print("Slice ID: {}".format(slice_id))

## Query Slices

In [ ]:
status, slices = orchestrator_proxy.slices(token=fabric_id_token)

print("Response Status {}".format(status))
print("Slices {}".format(slices))

## Slice Status

In [ ]:
import time

slice_state=None
while slice_state != 'StableOK':
    status, slice_status = orchestrator_proxy.slice_status(token=fabric_id_token, slice_id=slice_id)
    slice_state = slice_status.slice_state
    print("Slice State: {}".format(slices[0].slice_state))
    time.sleep(20)

#status, slice_status = orchestrator_proxy.slice_status(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Slice Status {}".format(slice_status))

## Query Slivers

In [ ]:
status, slivers = orchestrator_proxy.slivers(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Slivers {}".format(reservations))

## Sliver Status

In [ ]:
for sliver in slivers:
    sliver_id=sliver.reservation_id
    status, reservation_status = orchestrator_proxy.sliver_status(token=fabric_id_token, slice_id=slice_id, sliver_id=sliver_id)

    print("Response Status {}".format(status))
    print("Reservation Status {}".format(reservation_status))

## Configure the Experiment Nodes

In [ ]:
script= '#!/bin/bash  \n' \
        'lspci  \n'

In [ ]:
import paramiko 
#from scp import SCPClient, SCPException

key = paramiko.RSAKey.from_private_key_file("/home/fabric/.ssh/id_rsa")
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

for sliver in slivers:
    sliver_id=sliver.reservation_id
    node_name = sliver.name
    management_ip = sliver.management_ip
    
    print("Node {0} IP {1}".format(node_name, management_ip))
    
    client.connect(management_ip,username='ubuntu',pkey = key)

    stdin, stdout, stderr = client.exec_command('echo \"' + script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')
    #print (stdout.read())
    #print (stderr.read())
    print ('')
    print (str(stdout.read(),'utf-8').replace('\\n','\n'))
    #print ('stderr:')
    #print (str(stderr.read(),'utf-8').replace('\\n','\n'))

    client.close()

## Delete Slice

In [ ]:
#status, result = orchestrator_proxy.delete(token=fabric_id_token, slice_id=slice_id)
status, result = orchestrator_proxy.delete(token=fabric_id_token, slice_id='dcdb996a-9739-4cb5-b6ee-07753d5fd323')

print("Response Status {}".format(status))
print("Response received {}".format(result))